## Analizando frecuencia de palabras

Se realizan los siguientes pasos:

- Cargar dataframe de noticias.
- Cargar los cuerpos de noticia preprocesados.
- Encontrar la frecuencia de palabras clave de la categoría "Hecho contra la población civil" para cada cuerpo de noticia preprocesado.
- Crear el ranking de acuerdo a la suma de las frecuencias.

In [170]:
import numpy as np
import pandas as pd
import sqlite3
from nltk import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

def show_news(n, df):
    fila = df[df['ID']==n]
    print(
        "Suma:", fila['suma_freq'].values[0],
        "\nRanking:", fila['Ranking'].values[0],
        "\nTitulo:", fila['titulo'].values[0],
        "\nNumero palabras:", fila['long_cuerpo'].values[0],
        "\nDiario:", fila['diario'].values[0],
        "\nCuerpo:", fila['cuerpo'].values[0],
        "\nurl:", fila['url'].values[0],
        "\nID:", fila['ID'].values[0],
    )

-----------

**Cargando dataframe de noticias**
- Se encuentra la longitud del cuerpo de la noticia.
- Se asigna un ID a cada noticia.
- Se eliminan las noticias cuyo cuerpo no tiene ninguna palabra.

In [76]:
cnx = sqlite3.connect('../data/data.db')
df = pd.read_sql_query("SELECT * FROM news", cnx)
df['fecha_publicacion'] = pd.to_datetime(df['fecha_publicacion'])
df['long_cuerpo'] = df['cuerpo'].apply(lambda x: len(str(x).split()))
df['ID'] = df.index
df = df[df['long_cuerpo']!=0].reset_index()
df.head(2)

,index,titulo,cuerpo,fecha_publicacion,diario,url,page,long_cuerpo,ID
0,0,Campesino asesinado en Baraya,Un impacto en el pecho con una escopeta calibr...,2020-06-11 17:00:33,lanacion,https://www.lanacion.com.co/campesino-asesinad...,2,107,0
1,1,‘Morcilla’ a responder por homicidio de un jov...,Jorge Leonardo Pájaro Paredes conocido como ‘M...,2020-06-10 10:10:56,lanacion,https://www.lanacion.com.co/morcilla-a-respond...,2,228,1



**Cargando dataframe de cuerpo de noticia preprocesado**

- Se carga el archivo.
- Se pega al dataframe de noticias.

In [77]:
serie_word = pd.read_csv('serie_word.csv')['text']
print(serie_word.shape)
serie_word.head(2)

(19284,)


0    campesin asesin baray impact pech escopet cali...
1    morcill respond homicidi jov pajar conoc morci...
Name: text, dtype: object

In [ ]:
print("Tamanhos antes:", df.shape, serie_word.shape)
df['Preprocesado'] = serie_word.values

---------

**Encontrando la frecuencia de palabras clave de la categoría "Hecho contra la población civil" para cada cuerpo de noticia preprocesado**

- Se genera la lista de palabras clave y se preprocesa.
- Se eliminan las noticias que no tienen ninguna de las palabras clave.
- Se aplica el CountVectorizer sobre el dataframe de noticias para encontrar las frecuencias.

In [68]:
searchfor = ["homicidio",
             "masacre", 
             "secuestro",
             "tortura",
             "desplazamiento",
             "forzado",
             "confinamiento",
             "accidente",
             "minas",
             "antipersonal",
             "atentado",
             "reclutamiento",
             #"utilizacion NNA",
             "violencia",
             "sexual",
             "ataque",
             "indiscriminado",
             #"ataque",
             "bienes",
             "civiles"]

searchfor = pd.Series(searchfor)

spanishstemmer = SnowballStemmer('spanish')
print('Stemming words...')
# Stemming (word root)
searchfor = searchfor.apply(lambda x: " ".join(spanishstemmer.stem(word) for word in x.split()))
searchfor

Stemming words...


0         homicidi
1           masacr
2         secuestr
3           tortur
4          desplaz
5             forz
6           confin
7         accident
8              min
9     antipersonal
10           atent
11          reclut
12        violenci
13          sexual
14           ataqu
15     indiscrimin
16            bien
17           civil
dtype: object

In [78]:
searchfor_list = list(searchfor)
df['OK'] = df['Preprocesado'].str.contains('|'.join(searchfor_list))
df = df[df['OK'] == True]
print("Tamanhos despues:", df.shape)
df.head(2)

Tamanhos antes: (19284, 9) (19284,)
Tamanhos despues: (15115, 11)


,index,titulo,cuerpo,fecha_publicacion,diario,url,page,long_cuerpo,ID,Preprocesado,OK,ID3
0,0,Campesino asesinado en Baraya,Un impacto en el pecho con una escopeta calibr...,2020-06-11 17:00:33,lanacion,https://www.lanacion.com.co/campesino-asesinad...,2,107,0,campesin asesin baray impact pech escopet cali...,True,0
1,1,‘Morcilla’ a responder por homicidio de un jov...,Jorge Leonardo Pájaro Paredes conocido como ‘M...,2020-06-10 10:10:56,lanacion,https://www.lanacion.com.co/morcilla-a-respond...,2,228,1,morcill respond homicidi jov pajar conoc morci...,True,1


In [82]:
vectorizer = CountVectorizer(vocabulary=searchfor)
df['Freqs'] = df['Preprocesado'].apply(lambda x: vectorizer.fit_transform([x]).toarray())
df['Freqs'].head(2)

0    [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
1    [[5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
Name: Freqs, dtype: object

---------

**Creando el ranking de acuerdo a la suma de las frecuencias**

In [89]:
df['suma_freq'] = df.apply(lambda x: x['Freqs'].sum(), axis=1)
df['Ranking'] = df['suma_freq']/df['long_cuerpo']
df.head(2)

,index,titulo,cuerpo,fecha_publicacion,diario,url,page,long_cuerpo,ID,Preprocesado,OK,ID3,Freqs,suma_freq,Ranking
0,0,Campesino asesinado en Baraya,Un impacto en el pecho con una escopeta calibr...,2020-06-11 17:00:33,lanacion,https://www.lanacion.com.co/campesino-asesinad...,2,107,0,campesin asesin baray impact pech escopet cali...,True,0,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,0.009346
1,1,‘Morcilla’ a responder por homicidio de un jov...,Jorge Leonardo Pájaro Paredes conocido como ‘M...,2020-06-10 10:10:56,lanacion,https://www.lanacion.com.co/morcilla-a-respond...,2,228,1,morcill respond homicidi jov pajar conoc morci...,True,1,"[[5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",6,0.026316


In [153]:
aux = df[['suma_freq', 'ID']].reset_index()
aux.sort_values(by='suma_freq', ascending=False, inplace=True)
print(aux.head(2))
print(df[df['ID']==15539]['suma_freq'])

       index  suma_freq     ID
11942  15472         38  15539
14981  19118         25  19187
15472    38
Name: suma_freq, dtype: int64


In [171]:
show_news(15539, df)

Suma: 38 
Ranking: 0.023779724655819776 
Titulo: ¡Alto a la violencia contra las mujeres! 
Numero palabras: 1598 
Diario: lanacion 
Cuerpo: Carlos Daniel Mazabel Córdoba, secretario de Salud del Huila, aseguró que la violencia contra la mujer es un problema global de salud pública, con proporciones epidémicas.  El funcionario aseguró que en el ámbito departamental se reproducen diversas formas de violencia de género.  “Según datos de nuestro sistema de vigilancia epidemiológica, hasta el 30 de junio de 2013 se reportaron 1. 568 casos de violencia intrafamiliar y sexual, de los cuales 1. 347 se presentaron en mujeres.  De estos, 212 casos fueron de abuso sexual, 677 casos de violencia física, 388 de violencia psicológica y los 70 restantes fueron acoso y asalto sexual; todos presentados en mujeres”, subrayó.  Lina María Ramos, directora regional del Instituto de Medicina Legal y Ciencias Forenses, aseguró que de acuerdo con las actividades médico legales realizadas a mujeres víctimas de

In [180]:
show_news(19187, df)

Suma: 25 
Ranking: 0.02147766323024055 
Titulo: ¿Por qué las Farc continúan ensañadas con Pradera, Valle? 
Numero palabras: 1164 
Diario: elpais 
Cuerpo: Tras el atentado con motobomba, autoridades y analistas coinciden en que el municipio aún tiene un valor estratégico para la guerrilla.Pradera es el municipio del Valle del Cauca más afectado por la violencia de los grupos ilegales. En menos de dos años, esta ciudad, ubicada en las estribaciones de la cordillera Central, ha sido blanco de dos atentados terroristas con bombas que han dejado casi un centenar de heridos.  Una mezcla de la influencia de la columna Gabriel Galvis de las Farc y de dos poderosas bandas delincuenciales han creado el pánico en esta población, afectada no solo por los ataques guerrilleros, sino por las minas antipersonales sembradas en su área rural y por las extorsiones. El atentado más reciente ocurrió   el jueves pasado frente al edificio de la Alcaldía y a la estación de Policía de Pradera, cuando dos hombr

----------
**Verificando que el conteo de frecuencias haya sido correcto**

In [175]:
aux = df[df['ID']==15539]['Freqs'].values[0][0]
print(aux)

[ 0  0  0  0  2  0  0  0  0  0  0  0 27  7  0  0  1  1]


In [179]:
corpus = [df[df['ID']==15539]['Preprocesado'].values[0]]
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.toarray())
print(corpus[0].count('violenci'))

['homicidi', 'masacr', 'secuestr', 'tortur', 'desplaz', 'forz', 'confin', 'accident', 'min', 'antipersonal', 'atent', 'reclut', 'violenci', 'sexual', 'ataqu', 'indiscrimin', 'bien', 'civil']
[[ 0  0  0  0  2  0  0  0  0  0  0  0 27  7  0  0  1  1]]
27


-------
**Conclusiones preliminares**

- Las noticias con mayor ranking son las noticias más largas.
- No todas parecen servir para identificar un hecho contra la población civil.
- Hay que incluir algún tipo de peso para las palabras.